# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770376589210                   -0.53    8.0         
  2   -2.771686875282       -2.88       -1.32    1.0    194ms
  3   -2.771714123203       -4.56       -2.51    1.0    136ms
  4   -2.771714671052       -6.26       -3.28    2.0    227ms
  5   -2.771714710563       -7.40       -3.69    1.0    139ms
  6   -2.771714714937       -8.36       -4.61    1.0    143ms
  7   -2.771714715240       -9.52       -4.95    2.0    183ms
  8   -2.771714715249      -11.02       -5.90    1.0    150ms
  9   -2.771714715250      -12.38       -6.29    2.0    190ms
 10   -2.771714715250      -13.69       -7.17    1.0    162ms
 11   -2.771714715250   +  -15.05       -7.84    2.0    181ms
 12   -2.771714715250      -13.80       -8.72    1.0    153ms


-0.00013457370836600352

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770311748016                   -0.53    8.0         
  2   -2.771763399206       -2.84       -1.30    1.0    178ms
  3   -2.771800406676       -4.43       -2.67    1.0    140ms
  4   -2.771802065390       -5.78       -3.67    2.0    198ms
  5   -2.771802072605       -8.14       -3.94    2.0    202ms
  6   -2.771802074431       -8.74       -5.07    1.0    146ms
  7   -2.771802074476      -10.35       -5.73    2.0    201ms
  8   -2.771802074476      -12.25       -6.49    2.0    175ms
  9   -2.771802074476   +  -15.05       -6.85    1.0    176ms
 10   -2.771802074476      -13.69       -7.81    1.0    156ms
 11   -2.771802074476   +  -14.05       -8.55    2.0    201ms


0.017612220842216218

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457370836600352
Displaced dipole:  0.017612220842216218
Polarizability :   1.774679455058222


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920743250e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551369910e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852764832799e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694594427445e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088790312092e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.297832942182e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.351316318966e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.351316318966e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.048465609973e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 6.505534608321e-12
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.366855086991e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.366855086991e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.366995019067934e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.